In [2]:
import requests
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
import sys
import argparse
from PIL import Image, ImageDraw, ImageFont
from io import BytesIO
import json
import pprint
import cv2
import os
import time
import tensorflow as tf
from keras import layers, Input, models, optimizers
from keras.models import Model, Sequential
from keras.layers import MaxPooling2D, Conv2D,Activation, Dropout, Flatten, Dense
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# 1. 사용할 이미지 설정

img_dir ='C:/python_DA/img/test/015.jpg'                         # 학원컴퓨터
#img_dir ='C:/Jupyter_Script/img/test/test (103).jpg'            # 노트북

In [ ]:
# 2. 이미지 분리 및 속성 알려주는 함수 생성


def seperate_n_predict(img_dir):
    
    # 1) 사용할 model 설정
    model = load_model('C:/python_DA/프로젝트/model_pattern_정확도57_레이스없음.h5')    # 학원컴퓨터
    # model = load_model('C:/Jupyter_Script/파이널프로젝트/model/model_pattern_final.h5')   # 노트북
    model_json = model.to_json()
    
    
    # 2) 카카오 API 설정
    API_URL = 'https://kapi.kakao.com/v1/vision/product/detect'
    MYAPP_KEY = '89b7d96fc38f45fbf6ca52c2b13d2ad8'
    url = "https://kapi.kakao.com/v1/vision/product/detect"       
    headers = {'Authorization': 'KakaoAK {}'.format(MYAPP_KEY)}
    files = { 'file' : open(img_dir, 'rb')}
    response = requests.post(url, headers=headers, files=files)
    result = response.json()
    fig_w, fig_h = result['result']['width'], result['result']['height']
    
    # 2) 카카오 API 정보 저장
    info = []
    for each in result['result']['objects']:
        each = list(each.values())                                  # 순서 : x1, y1, x2, y2, class
        info.append(each)
    #print(info)

    # 3) 사진 자르기 
    for _ in range(len(info)):
        img = Image.open(img_dir)
        category = info[_][4]
        print(category)

    
        if category in ('shirts','t-shirts'):

            x = info[_][0]*fig_w - 100
            y = info[_][1]*fig_h - 100
            w = info[_][2]*fig_w + 50
            h = info[_][3]*fig_h + 50
#             path = 'C:/python_DA/img/cropped_img/top/'   
#             name = path + category + str(time.time()) + '.jpg'     
            img = img.crop((x,y,w,h))                               
            plt.imshow(img)
            img.show() 
        
            img = img.resize((150,150))
            data = np.array(img)
            data = data.astype('float')
            data = data/255
            data = data.reshape((1,150,150,3))

            var = ['camo', 'check', 'dot', 'floral', 'graphic', #'lace', 
                   'leopard', 'none', 'stripe']
            result = var[model.predict_classes(data)[0]]
            print(f'옷 : {category}')
            print(f'패턴 : {result}')        
        
        elif category in ('pants','skirts'): 

            x = info[_][0]*fig_w - 100
            y = info[_][1]*fig_h - 100
            w = info[_][2]*fig_w + 50
            h = info[_][3]*fig_h + 50
#             path = 'C:/python_DA/img/cropped_img/top/'   
#             name = path + category + str(time.time()) + '.jpg'     
            img = img.crop((x,y,w,h))                               
            plt.imshow(img)
            img.show() 

        
        # 4) 자른 사진 model 예측 
            img = img.resize((150,150))
            data = np.array(img)
            data = data.astype('float')
            data = data/255
            data = data.reshape((1,150,150,3))

            var = ['camo', 'check', 'dot', 'floral', 'graphic', #lace,
                   'leopard', 'none', 'stripe']
            result = var[model.predict_classes(data)[0]]
            print(f'옷 : {category}')
            print(f'패턴 : {result}')


### 배경제거

In [ ]:
# 1) 배경 제거 대상 이미지 설정
img = cv2.imread('./data/img/Abstract-Geo_Print_Mini_Skirt/img_00000033.jpg',1)
mask = np.zeros(img.shape[:2],np.uint8)
 
bgdModel = np.zeros((1,65),np.float64)
fgdModel = np.zeros((1,65),np.float64)
 
    
# 2) main 부분 좌표     
rect = (50,50,450,290) 
cv2.grabCut(img,mask,rect,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_RECT)
 
mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
img = img*mask2[:,:,np.newaxis]
 
plt.imshow(img),plt.colorbar(),plt.show()